In [1]:
import os

In [2]:
%pwd

'/home/saboa/projects/End-to-End-Kidney-Disease-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/saboa/projects/End-to-End-Kidney-Disease-Classification'

In [32]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_pretrained: bool
    params_batch_size: int
    params_device: str

In [33]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [34]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_pretrained=self.params.PRETRAINED,
            params_batch_size=self.params.BATCH_SIZE,
            params_device=self.params.DEVICE
        )

        return prepare_base_model_config

In [26]:
import os
import urllib.request as request
from zipfile import ZipFile
from torchvision import models
import torch
import torch.nn as nn
from torchsummary import summary

In [37]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = models.vgg16(pretrained=self.config.params_pretrained)

        summary(self.model, input_size=self.config.params_image_size, batch_size=self.config.params_batch_size, device=self.config.params_device)
        return self.model
    
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till):
        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for param in model.parameters()[:-freeze_till]:
                param.requires_grad = False

        n_inputs = model.classifier[6].in_features
        model.classifier[6] = nn.Sequential(
        nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.4),
        nn.Linear(256, classes), nn.LogSoftmax(dim=1))
        
        # prediction = tf.keras.layers.Dense(
        #     units=classes,
        #     activation="softmax"
        # )(flatten_in)

        # full_model = tf.keras.models.Model(
        #     inputs=model.input,
        #     outputs=prediction
        # )

        # full_model.compile(
        #     optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
        #     loss=tf.keras.losses.CategoricalCrossentropy(),
        #     metrics=["accuracy"]
        # )

        # full_model.summary()
        return model
    
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: torch.nn.Module):
        torch.save(model, path)

In [38]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-02-17 00:57:04,547: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-17 00:57:04,550: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-17 00:57:04,551: INFO: common: created directory at: artifacts]
[2024-02-17 00:57:04,552: INFO: common: created directory at: artifacts/prepare_base_model]


/home/saboa/projects/End-to-End-Kidney-Disease-Classification/kidney/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/saboa/projects/End-to-End-Kidney-Disease-Classification/kidney/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/saboa/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 88.8MB/s] 


TypeError: rand(): argument 'size' must be tuple of ints, but found element of type str at pos 2